## Local Import

* if you want to work localy with with half the data using individual pandas df <br\>*(my computer does not have sufficient local memory to join all tables)*

attempted stratagy (merge pd_df until memory bound then print to csv file then try again on csv file)

In [1]:
import json, csv
import pandas as pd

def JSON_to_dataFrame(file): 
    fp = open(file, encoding="utf8")
    df = pd.DataFrame([json.loads(x) for x in fp.readlines()])
    fp.close
    return df


In [26]:
# review_df = JSON_to_dataFrame("../Group_Project/Group1/review.json")
# business_df = JSON_to_dataFrame("../Group_Project/Group1/business.json")
# user_df = JSON_to_dataFrame("../Group_Project/Group1/user.json")
# tip_df = JSON_to_dataFrame("../Group_Project/Group1/tip.json")
# checkin_df = JSON_to_dataFrame("../Group_Project/Group1/checkin.json")

*********
## Merging Files
* paired the information down by combining the review business and user json files in to one merged csv file for processing (rbu_merged.csv) (deleted the other files and will be using this one to save space on my computer, if we need additional columns we will have to run this over adding the column)

In [66]:
# drop columns {cool, funny} cause I dont think we will use them
# review contains: {business_id, date, review_id, stars, text, useful, user_id}
# review_df = review_df.drop(['cool','funny'], axis=1)

In [61]:
# rename columns to identify the table
# review contains: {review_business_id, review_date, review_review_id, review_stars, review_text, review_useful, review_user_id}
# review_df = review_df.rename(columns={'business_id': 'review_business_id', 'date': 'review_date', 'review_id': 'review_review_id', 'stars': 'review_stars', 'text': 'review_text', 'useful': 'review_useful', 'user_id': 'review_user_id'})

In [67]:
# drop columns {postal_code, categories, address, attributes, categories, hours} 
# remaining fields: {business_id, city, is_open, latitude, longitude, name, review_count, stars, state}
# business_df = business_df.drop(['postal_code','categories', 'address', 'attributes', 'categories', 'hours'], axis=1)

In [64]:
# rename to reflect business table: {review_business_id, review_date, review_review_id, review_stars, review_text, review_useful, review_user_id}
# business contains: {business_business_id, business_city, business_is_open, business_latitude, business_longitude, business_name, business_review_count, business_stars, business_state}
# business_df = business_df.rename(columns={'business_id': 'business_business_id', 'city': 'business_city', 'is_open': 'business_is_open', 'latitude': 'business_latitude', 'longitude':'business_longitude','name':'business_name','review_count':'business_review_count','stars':'business_stars', 'state':'business_state'})

In [73]:
# rb_merge = pd.merge(review_df, business_df, left_on='review_business_id', right_on='business_business_id', how='inner')

In [55]:
# remaining fields: {business_id, city, is_open, latitude, longitude, name, review_count, stars, state}
# user_df = user_df.drop(['compliment_cool','compliment_cute','compliment_funny','compliment_hot','compliment_list','compliment_more','compliment_note','compliment_photos', 'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool', 'elite', 'funny'], axis=1)

In [71]:
# remaining fields: {user_average_stars', 'user_fans', 'user_friends', user_name', 'user_review_count', 'user_useful', 'user_user_id','user_yelping_since'}
# user_df = user_df.rename(columns={'average_stars': 'user_average_stars', 'fans': 'user_fans', 'friends': 'user_friends', 'name': 'user_name', 'review_count':'user_review_count','useful':'user_useful','user_id':'user_user_id','yelping_since':'user_yelping_since'})

In [74]:
# rbu_merge = pd.merge(rb_merge, user_df, left_on='review_user_id', right_on='user_user_id', how='inner')

In [76]:
# rbu_merge.to_csv(r'../Group_Project/Group1/rbu_merge.csv')

******
## Convert Merged File into DataFrame (using rbu_merge.csv)

In [16]:
try: 
    df = pd.read_csv("../Group_Project/Group1/rbu_merge.csv", dtype={'review_business_id':str, 'review_date':str, 'review_review_id':str, 'review_stars':float, 'review_text':str, 'review_useful':int, 'review_user_id':str, 'business_business_id':str, 'business_city':str, 'business_is_open':int, 'business_latitude':float, 'business_longitude':float, 'business_name':str, 'business_review_count':int, 'business_stars':float, 'business_state':str, 'user_average_stars':float, 'user_fans':int, 'user_friends':str, 'user_name':str, 'user_review_count':int, 'user_useful':int, 'user_user_id':str,'user_yelping_since':str})
except: 
    del()

In [28]:
df = df.drop('Unnamed: 0', axis=1)

In [27]:
df.isnull().sum()

Unnamed: 0                0
review_business_id        0
review_date               0
review_review_id          0
review_stars              0
review_text               1
review_useful             2
review_user_id            2
business_business_id      2
business_city             5
business_is_open          2
business_latitude         2
business_longitude        2
business_name             2
business_review_count     2
business_stars            2
business_state            2
user_average_stars        2
user_fans                 2
user_friends              2
user_name                12
user_review_count         4
user_useful               4
user_user_id              4
user_yelping_since        4
dtype: int64

****
## GCP Import
* login cradentaials provided to work with yelp_db database

In [ ]:
import sys
!{sys.executable} -m pip install PyMySQL

### MySql Server is nessisary to connect to GCP
* https://dev.mysql.com/downloads/mysql/

In [ ]:
import pymysql, os, sys, json
from IPython.display import clear_output
import multiprocessing as mp

In [ ]:
# read json file
file = "Group1/review.json"

json_data = open(file, encoding="utf8")
json_obj = [json.loads(x) for x in json_data.readlines()]


In [ ]:
num = len(json_obj)
print(num)

In [ ]:
# validate data before inserting
# TODO: check if string is the right type 
# TODO: check for duplicate data
# TODO: check for bad formats (date)

def validate_string(val): 
    if val != None: 
            if type(val) is int: 
                return str(val).encode('utf-8')
            else: 
                return val
            

### Processing takes to long so I used multiprocessing to speed up importing to host
* executing insert into gcp is atomic

this will take over 4 days to import one table!

In [ ]:
def insert_review(cursor, item, lock): 
          
    review_id = validate_string(item['review_id'])
    business_id = validate_string(item['business_id'])
    cool = validate_string(item['cool'])
    date = validate_string(item['date'])
    funny = validate_string(item['funny'])
    stars = validate_string(item['stars'])
    text = validate_string(item['text'])
    useful = validate_string(item['useful'])
    user_id = validate_string(item['user_id'])
    
    query = "INSERT INTO review(review_id,business_id,cool,date,funny,stars,text,useful,user_id) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    args = (review_id, business_id, cool, date, funny, stars, text, useful, user_id)
    lock.acquire()
    cursor.execute(query, args)
    lock.release()
    
    clear_output()
    cnt.value = cnt.value + 1
    print(cnt.value)
    
    

In [ ]:
# connect to GCP MySql db insert data into review table
con = pymysql.connect(host = '104.198.65.208', user='root', password = 'hackme', db = 'yelp_db')

cnt = mp.Value('i', 0)
lock = mp.Lock()
pool = mp.Pool()


try: 
    with con.cursor() as cursor:
        
        # parse json data to SQL
        # use map to update each row in GCP
        result = pool.map([ insert_review(cursor, item, lock) for item in json_obj ])
            
finally:
    con.commit()
    con.close()
    